# Applied Data Science Capstone - Week 3

## Part 1

### Import Libraries 

In [4]:
pip install lxml

     |████████████████████████████████| 5.5MB 6.0MB/s eta 0:00:01     |████████████████▋               | 2.9MB 6.0MB/s eta 0:00:01     |████████████████████████████▎   | 4.9MB 6.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import requests, six
import lxml.html as lh
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### Scrape Table Cells

In [6]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [7]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Parse Table Header

In [24]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    name = name.rstrip("\n")
    #print '%d:"%s"'%(i,name)
    col.append((name,[]))

In [25]:
col

[('Postal Code', []), ('Borough', []), ('Neighbourhood', [])]

### Creating Pandas DataFrame

In [26]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        data = data.rstrip("\n")
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [27]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [28]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [29]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [51]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)

df['Postal Code'].replace('', np.nan, inplace=True)
df.dropna(subset=['Postal Code'], inplace=True)

In [52]:
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [53]:
#result = df.groupby(['Postal Code']).agg(', '.join).reset_index()

In [61]:
#df.loc[df['Neighbourhood'] == 'Not assigned']

In [63]:
df.shape

(103, 3)

## Part 2

### Load CSV files to Python Pandas

In [66]:
# Read data from file 'Geospatial_Coordinates.csv' 
data = pd.read_csv("Geospatial_Coordinates.csv") 
# Preview the first 5 lines of the loaded data 
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [67]:
right_merged = pd.merge(df, data, how="right", on=["Postal Code"])
right_merged.head()
right_merged.shape

(103, 5)

In [68]:
right_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Part 3

In [72]:
neighborhoods = right_merged[right_merged['Borough'].str.contains("Toronto")]

In [86]:
neighborhoods

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [89]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


### Create a new dataframe of the Central Toronto data

In [91]:
toronto_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
del toronto_data['Postal Code']
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Davisville North,43.712751,-79.390197
2,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,Central Toronto,Davisville,43.704324,-79.388790
4,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


### Use geopy library to get the latitude and longitude values of Toronto City

In [92]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [93]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top

In [94]:
import folium # map rendering library

In [95]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [96]:
CLIENT_ID = 'ZCPEM44O3PECL2UP424KC12J2ZBY3ZSDWL3HY03EDI1DQDP2' # your Foursquare ID
CLIENT_SECRET = 'AT0ICKVJOUSXYKQWBYWLTLVYE03DKXJOBBMJMSZP00D5SV1M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZCPEM44O3PECL2UP424KC12J2ZBY3ZSDWL3HY03EDI1DQDP2
CLIENT_SECRET:AT0ICKVJOUSXYKQWBYWLTLVYE03DKXJOBBMJMSZP00D5SV1M


### Let's explore the first neighborhood in our dataframe

In [97]:
toronto_data.loc[0, 'Neighbourhood']

'Lawrence Park'

In [99]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


### Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters

In [100]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZCPEM44O3PECL2UP424KC12J2ZBY3ZSDWL3HY03EDI1DQDP2&client_secret=AT0ICKVJOUSXYKQWBYWLTLVYE03DKXJOBBMJMSZP00D5SV1M&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [101]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffb27b04c759760acf317e4'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72696303913755,
          'lng': -79.39438246708775}],
        'distance': 465,
        'c

In [102]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [104]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [105]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,HYC Design Inc.,Business Service,43.726793,-79.391681
2,Zodiac Swim School,Swim School,43.728532,-79.382860
3,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [106]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [108]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_

In [110]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville


In [112]:
print(toronto_venues.shape)
toronto_venues.head()

(111, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,HYC Design Inc.,43.726793,-79.391681,Business Service
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


In [113]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,33,33,33,33,33,33
Davisville North,9,9,9,9,9,9
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",1,1,1,1,1,1
"North Toronto West, Lawrence Park",21,21,21,21,21,21
Roselawn,4,4,4,4,4,4
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",15,15,15,15,15,15
"The Annex, North Midtown, Yorkville",20,20,20,20,20,20


In [114]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 63 uniques categories.


### Analyze Each Neighborhood

In [115]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Business Service,...,Sporting Goods Shop,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
toronto_onehot.shape

(111, 64)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [118]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Business Service,...,Sporting Goods Shop,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.030303,0.00,0.000000,0.000000,0.000000,0.030303,0.00,0.00,0.00,...,0.000000,0.0,0.000000,0.060606,0.00,0.030303,0.030303,0.00,0.000000,0.000000
1,Davisville North,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.00,0.00,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
2,"Forest Hill North & West, Forest Hill Road Park",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.000000,0.0,0.000000,0.250000,0.00,0.000000,0.000000,0.25,0.000000,0.000000
3,Lawrence Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.25,0.25,...,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.00,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.000000,1.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
5,"North Toronto West, Lawrence Park",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.047619,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.047619
6,Roselawn,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.066667,0.00,0.066667,0.066667,0.000000,0.000000,0.00,0.00,0.00,...,0.000000,0.0,0.066667,0.066667,0.00,0.000000,0.000000,0.00,0.066667,0.000000
8,"The Annex, North Midtown, Yorkville",0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.05,0.00,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000


In [119]:
toronto_grouped.shape

(9, 64)

### Let's print each neighborhood along with the top 5 most common venues

In [120]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0  Sandwich Place  0.09
1    Dessert Shop  0.09
2            Café  0.06
3     Coffee Shop  0.06
4     Pizza Place  0.06


----Davisville North----
              venue  freq
0             Hotel  0.11
1               Gym  0.11
2    Sandwich Place  0.11
3  Department Store  0.11
4              Park  0.11


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0                Trail  0.25
1     Sushi Restaurant  0.25
2        Jewelry Store  0.25
3   Mexican Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
              venue  freq
0  Business Service  0.25
1              Park  0.25
2       Swim School  0.25
3          Bus Line  0.25
4      Liquor Store  0.00


----Moore Park, Summerhill East----
                 venue  freq
0          Summer Camp   1.0
1  American Restaurant   0.0
2                  Pub   0.0
3       Ice Cream Shop   0.0
4    Indian Restaurant   0.0


----North Toronto West,  Law

### Let's put that into a _pandas_ dataframe

In [150]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [151]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Coffee Shop,Italian Restaurant,Pizza Place,Gym,Diner,Pharmacy
1,Davisville North,Department Store,Sandwich Place,Hotel,Food & Drink Shop,Park,Gym,Pizza Place,Gym / Fitness Center,Breakfast Spot,Bus Line
2,"Forest Hill North & West, Forest Hill Road Park",Trail,Mexican Restaurant,Sushi Restaurant,Jewelry Store,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Garden
3,Lawrence Park,Park,Bus Line,Swim School,Business Service,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Yoga Studio
4,"Moore Park, Summerhill East",Summer Camp,Yoga Studio,Dessert Shop,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Garden


### Cluster Neighborhoods

In [152]:
from sklearn.cluster import KMeans

In [153]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 1, 0, 0], dtype=int32)

In [154]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [161]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Davisville,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Coffee Shop,Italian Restaurant,Pizza Place,Gym,Diner,Pharmacy
1,0,Davisville North,Department Store,Sandwich Place,Hotel,Food & Drink Shop,Park,Gym,Pizza Place,Gym / Fitness Center,Breakfast Spot,Bus Line
2,0,"Forest Hill North & West, Forest Hill Road Park",Trail,Mexican Restaurant,Sushi Restaurant,Jewelry Store,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Garden
3,0,Lawrence Park,Park,Bus Line,Swim School,Business Service,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Yoga Studio
4,2,"Moore Park, Summerhill East",Summer Camp,Yoga Studio,Dessert Shop,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Garden
5,0,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Gift Shop,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Cosmetics Shop
6,1,Roselawn,Health & Beauty Service,Home Service,Ice Cream Shop,Garden,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gift Shop
7,0,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,American Restaurant,Vietnamese Restaurant,Pizza Place,Pub,Restaurant,Sandwich Place,Light Rail Station,Fried Chicken Joint,Liquor Store
8,0,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Coffee Shop,History Museum,Indian Restaurant,Donut Shop,Liquor Store,Middle Eastern Restaurant,Park,Pharmacy


In [156]:
toronto_merged = toronto_data
toronto_merged.rename(columns={'Neighbourhood': 'Neighborhood'})

,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Davisville North,43.712751,-79.390197
2,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,Central Toronto,Davisville,43.704324,-79.388790
4,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
6,Central Toronto,Roselawn,43.711695,-79.416936
7,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
8,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
